# Table of contents
1. [Purpose](#purpose)
2. [Inputs](#inputs)
    * [2.1 SQL Databases](#subinput1)
3. [Outputs](#outputs)
4. [Constraints](#constraints)
5. [Dependencies](#dependencies)
    * [5.1 Code Language](#codelanguage)
      * [5.1.1 package requirements](#packagerequirements)

6. [Code](#code)
7. [Tests](#tests)
8. [Provenance](#provenance)
9. [Citations](#citations)
---

# GAP Species Habitat Associations and the National Vegetation Classification (NVC) System Hierarchy
<br/>

## Purpose <a name="purpose"></a>
The U.S. Geological Survey Gap Analysis Project (GAP) created habitat maps for 1590 terrestrial vertebrate species throughout the conterminous US (CONUS), including 219 sub-species. Each species was associated with one or more land cover types represented in the National GAP Land Cover Map (McKerrow et al. 2018, Gergely et al. 2019). The USGS GAP Land Cover Map used vegetation classes based on NatureServe’s Ecological Systems Classification (Comer et al. 2003) and land cover classes described in the National Land Cover Dataset (Homer et al. 2007). These data described vegetation communities at a level of thematic detail useful for ascribing habitat types for species. A crosswalk of that map legend with the nested hierarchy of the US National Vegetation Classification (NVC) permits the assessment of species habitat extents throughtout a broadening thematic vegetation classification.

This notebook specifically outlines using the crosswalk of species habitat associations in the GAP land cover map with NVC macrogroups for linking USDA Forest Inventory and Analysis (FIA) plot level forest composition and structure data.

## Inputs <a name="inputs"></a>
The code in this notebook is a first step in combining GAP species habitat association information with FIA plot level data wherein forest structure and composition are tied to NVC macrogroup classes. Since the GAP habitat maps were developed using deductive methods (McKerrow et al. 2018, Gergely et al. 2019) which include landscape attributes other than land cover, it was necessary to identify species whose associations include forest and no other ancillary data such as distance to water or elevation limits.
    
Data compiled by code in this notebook are assembled in a wildlife habitat relationship database (WHRdb). The WHRdb was queried to compile a list of species associated with at least one forested land cover type (referred to as <b>map units</b> in WHRdb tables) and no ancillary landscape attributes. Modeling parameters were assembled for each species by seasonality - summer (breeding for migratory birds), winter, and year-round (habitat utilized during both summer and winter). Additionally, modeling regions were used to stratify the CONUS into six regions (Northwest, Southwest, Great Plains, Upper Midwest, Southeast, and Northeast). These regions allowed for efficient processing of the species distribution models on smaller, ecologically homogenous extents. Therefore, species habitat associations may occur in multiple seasons and/or across multiple regions. A species model code was developed to maintain unique season/region combinations.

* **U.S. Geological Survey Gap Analysis Project (GAP) Wildlife Habitat Relationship Database** <a name="subinput1"></a>  
Information about access to a given instance of this database is pending. The database is described here: https://www.sciencebase.gov/catalog/item/527d0a83e4b0850ea0518326.

Additionally, this code utilizes an SQL server database that was assembled by intersecting numerous spatial data layers including the PAD-US, species habitat maps, boundary layers such as states, counties, LCCs, ecoregions, etc.

* **U.S. Geological Survey Gap Analysis Project (GAP) Analytical Database** <a name="subinput1"></a>  
These analyses utilize the GAP Analytic database developed by Linda Schueck, Anne Davidson, and Leah Dunn at Boise State University. Information about access to a given instance of this database is pending. The database is described here: https://www.sciencebase.gov/catalog/item/5b736251e4b0f5d5787c61df.

At the time this workflow was developed, both databases were available on a local server. Hence, all code references a local instance of these databases. This code is only replicable given access to a local instance of these databases.

This analysis relies on three major datasets within the analytic database including:
1. The detailed land cover dataset: GAP/LANDFIRE National Terrestrial Ecosystems 2011: National Inventory of Vegetation and Land Use  (https://www.sciencebase.gov/catalog/item/5810cd6fe4b0f497e7975237)
2. The U.S. National Vegetation Classification Ver 2.0 (http://usnvc.org/explore-classification/)
3. A crosswalk between Ecological Systems and the USNVC Types (Embedded in the Attribute file for the National Terrestrials Ecosystems data https://www.sciencebase.gov/catalog/item/5810cd6fe4b0f497e7975237.

## Outputs <a name="outputs"></a>
A table and output CSV file with data on species map unit and NVC macrogroup associations organized by season and modeling region. The following columns are included in the table:
* <b>ScientificName</b> - species scientific name
* <b>CommonName</b> - species common name
* <b>SC</b> - species 6-letter code
* <b>SMC</b> - species 9-charcter model code
* <b>MUCode</b> - land cover map unit 4-digit code
* <b>MUName</b> - land cover map unit name
* <b>Season</b> - species model season
* <b>Region</b> - species model region
* <b>Macrogroup</b> - NVC macrogroup name


## Constraints <a name="constraints"></a>
The Land Cover data attempts to map as accurately as possible the fine vegetative details of the nation’s vegetation. However, there are some limitations to the data that users should keep in mind. The data set uses a 30 meter pixel cell and in most areas a minimum mapping unit of 0.4 ha (1 acre). This means that small patches of vegetation can be missed in the modeling process.  The USNVC is a dynamic content standard, reflecting the state of the science at any point in time; therefore this analysis represents the conservation assessment based on the types that were named and described in the 2017 release of the classification.  

## Dependencies <a name="dependencies"></a>
These analyses are currently reliant on a local instance of the GAP Wildlife Habitat Relationship database (GAP WHRdb) and the GAP Analytical Database (GAP analytic db).

Code execution dependencies

* **Code Language** <a name="codelanguage"></a>  
This code is written in Python 3.x. Earlier versions may be compatible given slight alterations in syntax and package components.
  * **Python package requirements** <a name="packagerequirements"></a>  
  The required packages for proper code execution inlclude:
    - Pandas version 0.2x or above
    - pyodbc version 4.x or above


## Tests <a name="tests"></a>
In developing this code the results were compared against a dataset generated through an independent spatial analysis.  


## Provenance <a name="provenance"></a>

## Code <a name="code"></a>

#### Import required Python packages

In [14]:
import pyodbc
import pandas as pd
import pandas.io.sql as psql
#%matplotlib notebook

#### Accessing the local SQL server instance of the GAP Analytic database

In [15]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#            ++++ Directory Locations ++++
workDir = 'C:/Data/USGS Analyses/Species-Habitat-NVC-Associations/Scripts/'


#############################################################################################
################################### LOCAL FUNCTIONS #########################################
#############################################################################################


## --------------Cursor and Database Connections--------------------

def ConnectToDB(connectionStr):
    '''
    (str) -> cursor, connection

    Provides a cursor within and a connection to the database

    Argument:
    connectionStr -- The SQL Server compatible connection string
        for connecting to a database
    '''
    try:
        con = pyodbc.connect(connectionStr)
    except:
        connectionStr = connectionStr.replace('11.0', '10.0')
        con = pyodbc.connect(connectionStr)

    return con.cursor(), con

## ----------------Database Connection----------------------

def DBConnection(dbname):
    '''
    Returns a cursor and connection within the GAP analytic database.
    '''
    # Database connection parameters
    dbstr = """DRIVER=SQL Server Native Client 11.0;
                    SERVER=CHUCK\SQL2014;
                    UID=;
                    PWD=;
                    TRUSTED_CONNECTION=Yes;
                    DATABASE={0};"""
    dbConStr = dbstr.format(dbname)

    return ConnectToDB(dbConStr)

#############################################################################################
#############################################################################################
#############################################################################################


In [16]:
## Connect to the Analytic Database
print("+"*45)
print("Connecting to Database ....")
cur, conn = DBConnection('GAP_AnalyticDB')

+++++++++++++++++++++++++++++++++++++++++++++
Connecting to Database ....


#### Generating an SQL string to pass to a pandas dataframe

In [17]:
# Make an SQL that pulls out NVC macrogroups from the Analytic db
sqlGA = """SELECT
            level3,
            nvc_macro AS Macrogroup
        FROM
            GAP_AnalyticDB.dbo.gap_landfire
        WHERE level3 > 0"""


#### Passing the SQL and getting a pandas dataframe

In [18]:
# Make a dataframe of the macrogroups with the 4-digit code
print("Creating Dataframe of NVC Macrogroups from Gap Analytic Datbase ....")
dfMacro = psql.read_sql(sqlGA, conn)

Creating Dataframe of NVC Macrogroups from Gap Analytic Datbase ....


In [19]:
dfMacro

,level3,Macrogroup
0,9238,Caribbean Swamp
1,9702,Caribbean Swamp
2,9305,Caribbean Swamp
3,9236,Western Atlantic & Caribbean Mangrove
4,4132,Caribbean Coastal Lowland Dry Forest
5,4135,Caribbean Coastal Lowland Dry Forest
6,4134,Caribbean Coastal Lowland Dry Forest
7,4145,Longleaf Pine Woodland
8,4505,Longleaf Pine Woodland
9,4506,Longleaf Pine Woodland


#### Delete the cursor and connection to the Analytic database to reuse variables for connection to WHR database

In [20]:
# Delete the previous db connection variables and reset them for
# connecting to the WHR database
del cur, conn
print('Connecting to Gap WHR Database ....')
cur, conn = DBConnection('GapVert_48_2001')

Connecting to Gap WHR Database ....


#### Pull out species map unit associations from the WHR for species associated with at least one forested type and no ancillary constraints

In [21]:
# Make an SQL that gets data for species who have at least one forested map unit
# and have NO ancillary data constraints
sqlWHR = """WITH
-- Build table of species seasonal/regional use of ancillary data
smAnc AS (
	SELECT	i.strUC AS strUC
		  ,	a.strSpeciesModelCode AS strSpeciesModelCode
		  , CAST(ysnHandModel AS int) AS intHandModel
		  , CAST(ysnHydroFW AS int) AS intHydroFW
		  , CAST(ysnHydroOW AS int) AS intHydroOW
		  , CAST(ysnHydroWV AS int) AS intHydroWV
		  , CASE
				WHEN (strSalinity Is Null OR strSalinity = 'All Types') THEN 0
				ELSE 1
			END AS intSalinity
		  , CASE
				WHEN (strStreamVel Is Null OR strStreamVel = 'All Types') THEN 0
				ELSE 1
			END AS intStreamVel
		  , CASE
				WHEN strEdgeType Is Null THEN 0
				ELSE 1
			END AS intEdgeType
		  , CASE
				WHEN strUseForInt Is Null THEN 0
				ELSE 1
			END AS intUseForInt
		  , CAST(cbxContPatch AS int) AS intContPatch
		  ,	CAST(cbxNonCPatch AS int) AS intNonCPatch
		  , CASE
				WHEN intPercentCanopy Is Null THEN 0
				ELSE 1
			END AS intPercentCanopy 
		  , CASE
				WHEN intAuxBuff Is Null THEN 0
				ELSE 1
			END AS intAuxBuff
		  , CASE
				WHEN strAvoid Is Null THEN 0
				ELSE 1
			END AS intAvoid
		  ,	CAST(ysnUrbanExclude AS int) AS intUrbanExclude
		  ,	CAST(ysnUrbanInclude AS int) AS intUrbanInclude
		  , CASE
				WHEN (intElevMin Is Null OR intElevMin < 1) THEN 0
				ELSE 1
			END AS intElevMin
		  , CASE
				WHEN intElevMax Is Null THEN 0
				ELSE 1
			END AS intElevMax

	FROM GapVert_48_2001.dbo.tblModelAncillary a 
		 INNER JOIN GapVert_48_2001.dbo.tblModelInfo i
			ON a.strSpeciesModelCode = i.strSpeciesModelCode
	WHERE	i.ysnIncludeSubModel = 1 
	),

/*
	Identify species ancillary data use across seasonal/regional submodels.
	This sums up how many submodels have at least one ancillary parameter selection.
	Return only species whose submodel ancillary tally total is 0 meaning NO
	submodels use ANY ancillary parameter
*/
NonAncillary AS (
	SELECT
		strUC
	FROM smAnc
	GROUP BY strUC
	HAVING
		SUM ( intHandModel +
			  intHydroFW +
			  intHydroOW +
			  intHydroWV +
			  intSalinity +
			  intStreamVel +
			  intEdgeType +
			  intUseForInt +
			  intContPatch +
			  intNonCPatch +
			  intPercentCanopy +
			  intAuxBuff +
			  intAvoid +
			  intUrbanExclude +
			  intUrbanInclude +
			  intElevMin +
			  intElevMax ) = 0

	),

/*
	Pull out species whose models include at least one map unit selection
	that is a forested map unit.
	NOTE: This criterion is only for primary map units. Secondary map units are ignored
	Also, include only those species for whom there are valid submodels.
*/
ForestSelected AS 
(SELECT
		tblMapUnitDesc.intLSGapMapCode,
		tblMapUnitDesc.strLSGapName,
		tblMapUnitDesc.intForest,
		tblSppMapUnitPres.strSpeciesModelCode,
		tblSppMapUnitPres.ysnPres,
		tblModelInfo.ysnIncludeSubModel,
		tblModelInfo.strUC
FROM 
		tblMapUnitDesc FULL JOIN tblSppMapUnitPres 
		ON tblMapUnitDesc.intLSGapMapCode = tblSppMapUnitPres.intLSGapMapCode
		INNER JOIN tblModelInfo 
		ON tblSppMapUnitPres.strSpeciesModelCode = tblModelInfo.strSpeciesModelCode
WHERE 
		tblMapUnitDesc.intForest = 1 AND 
		tblSppMapUnitPres.ysnPres = 1 AND 
		tblSppMapUnitPres.strSpeciesModelCode Not Like '%m_' AND
		tblModelInfo.ysnIncludeSubModel = 1),

/*
	Pull out the scientific and common names from the taxa table
*/
Taxa AS
(SELECT strUC, strSciName, strComName
FROM tblTaxa)

/*
	Combine the non-ancillary, at least one forested map unit selection, and
	corresponding taxa scientific name and common name sub-queries into final output

*/


SELECT 	Taxa.strSciName AS ScientificName,
		Taxa.strComName AS CommonName,
		Taxa.strUC AS SC,
		strSpeciesModelCode AS SMC,
		ForestSelected.intLSGapMapCode AS MUCode,
		ForestSelected.strLSGapName AS MUName,
		CASE 
			WHEN 
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 8, 1)='y'
			  THEN 'year-round'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 8, 1)='s'
			  THEN 'summer'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 8, 1)='w'
			  THEN 'winter'
		END AS Season,
		
		CASE 
			WHEN 
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 9, 1)='1'
			  THEN 'Northwest'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 9, 1)='2'
			  THEN 'Upper Midwest'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 9, 1)='3'
			  THEN 'Northeast'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 9, 1)='4'
			  THEN 'Southwest'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 9, 1)='5'
			  THEN 'Great Plains'
			WHEN
			  SUBSTRING(ForestSelected.strSpeciesModelCode, 9, 1)='6'
			  THEN 'Southeast'
		END AS Region

FROM
	ForestSelected INNER JOIN NonAncillary
	ON
	ForestSelected.strUC = NonAncillary.strUC
	INNER JOIN Taxa
	ON ForestSelected.strUC = Taxa.strUC"""

#### Make a dataframe using the above criteria

In [22]:
# Make a dataframe of the forest/non-ancillary species map unit associations
dfSppMUs = psql.read_sql(sqlWHR, conn)

In [23]:
dfSppMUs

,ScientificName,CommonName,SC,SMC,MUCode,MUName,Season,Region
0,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9208,Southern Coastal Plain Seepage Swamp and Bayga...,year-round,Southeast
1,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9703,Southern Coastal Plain Nonriverine Cypress Dom...,year-round,Southeast
2,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9715,Southern Piedmont/Ridge and Valley Upland Depr...,year-round,Southeast
3,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9801,Atlantic Coastal Plain Blackwater Stream Flood...,year-round,Southeast
4,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9838,Southern Coastal Plain Hydric Hammock ...,year-round,Southeast
5,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9841,Southern Piedmont Small Floodplain and Riparia...,year-round,Southeast
6,Storeria victa,Florida Brownsnake,rFLBRx,rFLBRx-y6,9852,Southern Coastal Plain Blackwater River Floodp...,year-round,Southeast
7,Plethodon aureolus,Tellico Salamander,aTESAx,aTESAx-y6,4127,Central and Southern Appalachian Montane Oak F...,year-round,Southeast
8,Sorex haydeni,Prairie Shrew,mPRSHx,mPRSHx-y5,9818,Central Interior and Appalachian Floodplain Sy...,year-round,Great Plains
9,Sorex haydeni,Prairie Shrew,mPRSHx,mPRSHx-y2,9818,Central Interior and Appalachian Floodplain Sy...,year-round,Upper Midwest


### Merge the two dataframes derived from the Analytic and WHR databases to show map units and their associated macrogroups

In [24]:
# Merge the dataframes from the WHR and Analytc dbs using the columns
# that have map unit 4-digit codes: MUCode and level3 respectively
dfSppMUs_Macro = pd.merge(left=dfSppMUs, right=dfMacro, how='inner',
                      left_on='MUCode', right_on='level3')

In [25]:
dfSppMUs_Macro=dfSppMUs_Macro.sort_values(by=['SMC'])
dfSppMUs_Macro

,ScientificName,CommonName,SC,SMC,MUCode,MUName,Season,Region,level3,Macrogroup
9529,Aneides lugubris,Arboreal Salamander,aARSAx,aARSAx-y1,4317,Mediterranean California Lower Montane Black O...,year-round,Northwest,4317,Californian Forest & Woodland
9530,Aneides lugubris,Arboreal Salamander,aARSAx,aARSAx-y4,4317,Mediterranean California Lower Montane Black O...,year-round,Southwest,4317,Californian Forest & Woodland
8835,Plethodon caddoensis,Caddo Mountain Salamander,aCMSAx,aCMSAx-y6,4122,Ouachita Montane Oak Forest ...,year-round,Southeast,4122,Southern & South-Central Oak - Hickory - Pine ...
3737,Plethodon caddoensis,Caddo Mountain Salamander,aCMSAx,aCMSAx-y6,4115,Ozark-Ouachita Dry-Mesic Oak Forest ...,year-round,Southeast,4115,Southern & South-Central Oak - Hickory - Pine ...
3911,Plethodon caddoensis,Caddo Mountain Salamander,aCMSAx,aCMSAx-y6,4207,Ozark-Ouachita Mesic Hardwood Forest ...,year-round,Southeast,4207,Central Midwest Mesic Forest
5159,Plethodon kentucki,Cumberland Plateau Salamander,aCPSAx,aCPSAx-y6,4401,Southern and Central Appalachian Cove Forest ...,year-round,Southeast,4401,Appalachian-Central Interior-Northeastern Mesi...
1618,Plethodon kentucki,Cumberland Plateau Salamander,aCPSAx,aCPSAx-y6,4402,South-Central Interior Mesophytic Forest ...,year-round,Southeast,4402,Appalachian-Central Interior-Northeastern Mesi...
9491,Batrachoseps attenuatus,California Slender Salamander,aCSLSx,aCSLSx-y1,4606,North Pacific Maritime Mesic-Wet Douglas-fir-W...,year-round,Northwest,4606,Vancouverian Lowland & Montane Forest
967,Batrachoseps attenuatus,California Slender Salamander,aCSLSx,aCSLSx-y1,4545,California Coastal Closed-Cone Conifer Forest ...,year-round,Northwest,4545,Californian Forest & Woodland
6506,Batrachoseps attenuatus,California Slender Salamander,aCSLSx,aCSLSx-y1,4607,North Pacific Mesic Western Hemlock-Silver Fir...,year-round,Northwest,4607,Vancouverian Lowland & Montane Forest


### Export the data to a CSV file

In [26]:
# Export to CSV file
dfSppMUs_Macro.to_csv(workDir + "Species-Habitat-Macrogroups.csv")

## Citations <a name="citations"></a>
Comer, P.J., Faber-Langendoen D., Evans, R., Gawler, S.C., Josse, C., Kittel, G., Menard, S., Pyne, M., Reid, M., Schulz, K., Snow, K., and Teague, J., 2003, Ecological systems of the United States—A working classification of U.S. terrestrial systems: Arlington, NatureServe. [Also available at http://www.natureserve.org/biodiversity-science/publications/ecological-systems-united-states.

Gergely, K.J., Boykin, K.G., McKerrow, A.J., Rubino, M.J., Tarr, N.M., and Williams, S.G., 2019, Gap Analysis Project (GAP) terrestrial vertebrate species richness maps for the conterminous U.S.: U.S. Geological Survey Scientific Investigations Report 2019–5034, 99 p., https://doi.org/10.3133/sir20195034.

Homer, C.J., Dewitz, J., Fry, M., Coan, N., Hossain, C., Larson, N., Herold, A., McKerrow, J.N., VanDriel, J.N., and Wickham, J., 2007, Completion of the 2001 National Land Cover Database for the Conterminous United States: Photogrammetric Engineering and Remote Sensing, v. 73, issue 4, p. 337–341, accessed July 30, 2018, at https://www.researchgate.net/profile/Collin_Homer/publication/237239863_Completion_of_the_2001_National_Land_Cover_Database_for_the_Conterminous_United_States/links/0046352ab204d93a3e000000.pdf.

McKerrow, A.J., Tarr, N.M., Rubino, M.J., and Williams, S.G., 2018, Patterns of species richness hotspots and estimates of their protection are sensitive to spatial resolution: Diversity and Distributions, v. 24, issue 10, p. 1464–1477, accessed July 30, 2018, at https://doi.org/10.1111/ddi.12779.

U.S. Geological Survey Gap Analysis Program, 20160513, GAP/LANDFIRE National Terrestrial Ecosystems 2011: U.S. Geological Survey, https://doi.org/10.5066/F7ZS2TM0.

USNVC [United States National Vegetation Classification]. 2017. United States National Vegetation Classification Database, V2.01. Federal Geographic Data Committee, Vegetation Subcommittee, Washington DC. [usnvc.org/explore-classification/] (accessed 30 March 2017).